### Problem Statement

1) Prepare a classification model using SVM for salary data 

Data Description:

age -- age of a person
workclass	-- A work class is a grouping of work 
education	-- Education of an individuals	
maritalstatus -- Marital status of an individulas	
occupation	 -- occupation of an individuals
relationship -- 	
race --  Race of an Individual
sex --  Gender of an Individual
capitalgain --  profit received from the sale of an investment	
capitalloss	-- A decrease in the value of a capital asset
hoursperweek -- number of hours work per week	
native -- Native of an individual
Salary -- salary of an individual


In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA

In [127]:
salary_train = pd.read_csv('SalaryData_Train(1).csv')
salary_test  = pd.read_csv('SalaryData_Test(1).csv')

In [128]:
salary_test.shape, salary_train.shape

((15060, 14), (30161, 14))

In [129]:
salary_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30161 entries, 0 to 30160
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   age            30161 non-null  int64 
 1   workclass      30161 non-null  object
 2   education      30161 non-null  object
 3   educationno    30161 non-null  int64 
 4   maritalstatus  30161 non-null  object
 5   occupation     30161 non-null  object
 6   relationship   30161 non-null  object
 7   race           30161 non-null  object
 8   sex            30161 non-null  object
 9   capitalgain    30161 non-null  int64 
 10  capitalloss    30161 non-null  int64 
 11  hoursperweek   30161 non-null  int64 
 12  native         30161 non-null  object
 13  Salary         30161 non-null  object
dtypes: int64(5), object(9)
memory usage: 3.2+ MB


In [130]:
salary_train.describe().T

,count,mean,std,min,25%,50%,75%,max
age,30161.0,38.438115,13.134830,17.0,28.0,37.0,47.0,90.0
educationno,30161.0,10.121316,2.550037,1.0,9.0,10.0,13.0,16.0
capitalgain,30161.0,1092.044064,7406.466611,0.0,0.0,0.0,0.0,99999.0
capitalloss,30161.0,88.302311,404.121321,0.0,0.0,0.0,0.0,4356.0
hoursperweek,30161.0,40.931269,11.980182,1.0,40.0,40.0,45.0,99.0


In [131]:
salary_train.duplicated().sum()

3258

In [132]:
salary_train.drop_duplicates(inplace = True, ignore_index=True)

In [133]:
for col in salary_train.columns:
    if salary_train[col].dtypes == 'object':
        print(f'{col} : {salary_train[col].unique()}')

workclass : [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' Self-emp-inc' ' Without-pay']
education : [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' 7th-8th' ' Doctorate' ' Assoc-voc' ' Prof-school'
 ' 5th-6th' ' 10th' ' Preschool' ' 12th' ' 1st-4th']
maritalstatus : [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
occupation : [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Transport-moving' ' Farming-fishing'
 ' Machine-op-inspct' ' Tech-support' ' Craft-repair' ' Protective-serv'
 ' Armed-Forces' ' Priv-house-serv']
relationship : [' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
race : [' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
sex : [' Male' ' Female']
native : [' United-States' ' Cuba' ' Jamaica' ' India' ' Mexico' ' Puerto-R

In [134]:
salary_train.Salary.value_counts()

 <=50K    20023
 >50K      6880
Name: Salary, dtype: int64

In [135]:
data = pd.get_dummies(data = salary_train, columns = ['workclass','education','maritalstatus',
       'occupation', 'relationship', 'race', 'sex', 'native', 'Salary'], drop_first= True)
data

,age,educationno,capitalgain,capitalloss,hoursperweek,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,native_ Puerto-Rico,native_ Scotland,native_ South,native_ Taiwan,native_ Thailand,native_ Trinadad&Tobago,native_ United-States,native_ Vietnam,native_ Yugoslavia,Salary_ >50K
0,39,13,2174,0,40,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,50,13,0,0,13,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,38,9,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,53,7,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,28,13,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26898,53,14,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
26899,22,10,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26900,27,12,0,0,38,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26901,58,9,0,0,40,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


### Feature Scalling

In [136]:
scale = MinMaxScaler()
data.iloc[:, :5] = pd.DataFrame(scale.fit_transform(data.iloc[:, :5]))
data

,age,educationno,capitalgain,capitalloss,hoursperweek,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,native_ Puerto-Rico,native_ Scotland,native_ South,native_ Taiwan,native_ Thailand,native_ Trinadad&Tobago,native_ United-States,native_ Vietnam,native_ Yugoslavia,Salary_ >50K
0,0.301370,0.800000,0.021740,0.0,0.397959,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,0.452055,0.800000,0.000000,0.0,0.122449,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0.287671,0.533333,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.493151,0.400000,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0.150685,0.800000,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26898,0.493151,0.866667,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
26899,0.068493,0.600000,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26900,0.136986,0.733333,0.000000,0.0,0.377551,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
26901,0.561644,0.533333,0.000000,0.0,0.397959,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [137]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
X.shape, y.shape

((26903, 94), (26903,))

### train test split

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0, stratify=y)

In [139]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((21522, 94), (5381, 94), (21522,), (5381,))

### Feature Selection

In [140]:
# get best 10 features selectKbest
bestfeatures = SelectKBest(score_func = chi2, k=20)
fit1 = bestfeatures.fit(X_train, y_train)

In [141]:
features = pd.DataFrame({'features' : X.columns,
                        'Score' : fit1.scores_})
features.sort_values(by = 'Score', ascending=False).head(20)

,features,Score
27,maritalstatus_ Married-civ-spouse,2352.675582
29,maritalstatus_ Never-married,1401.485164
47,relationship_ Own-child,873.830128
34,occupation_ Exec-managerial,788.335097
45,relationship_ Not-in-family,583.729907
22,education_ Masters,577.019214
40,occupation_ Prof-specialty,575.817400
38,occupation_ Other-service,550.982563
19,education_ Bachelors,542.930870
2,capitalgain,540.995358


In [142]:
feat_ind = features.sort_values(by = 'Score', ascending=False).head(5).index
feat_ind 

Int64Index([27, 29, 47, 34, 45], dtype='int64')

In [143]:
new_X_train = X_train.iloc[:, feat_ind]
new_X_train

,maritalstatus_ Married-civ-spouse,maritalstatus_ Never-married,relationship_ Own-child,occupation_ Exec-managerial,relationship_ Not-in-family
14488,1,0,0,1,0
5712,0,1,0,0,1
10501,0,1,1,0,0
847,1,0,0,1,0
24768,0,0,0,0,1
...,...,...,...,...,...
2323,1,0,0,1,0
3578,1,0,0,0,0
22295,1,0,0,1,0
12722,0,0,0,1,1


In [144]:
new_X_test = X_test.iloc[:, feat_ind]
new_X_test

,maritalstatus_ Married-civ-spouse,maritalstatus_ Never-married,relationship_ Own-child,occupation_ Exec-managerial,relationship_ Not-in-family
10182,1,0,0,1,0
17706,0,0,0,0,0
6656,0,1,0,0,1
18575,0,1,1,0,0
16057,1,0,0,0,0
...,...,...,...,...,...
25398,0,1,0,0,1
23254,0,0,0,0,0
17083,1,0,0,0,0
4555,1,0,0,0,0


### Hyperparameter tuning

In [145]:
#estimator = SVC()
#param_grid = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
#             'gamma'   : ['scale', 'auto'],
#             'C'      : [0.01, 0.1, 1, 1.0, 10]
#             }
#grid = GridSearchCV(estimator = estimator, param_grid = param_grid,cv = 5)
#grid.fit(new_X_train, y_train)

In [146]:
#grid.best_params_

In [147]:
#grid.best_score_

### Model Building

In [148]:
clf = SVC(class_weight='balanced')
clf.fit(new_X_train, y_train)
y_pred = clf.predict(new_X_test)

In [149]:
y_pred

array([1, 0, 0, ..., 1, 1, 0], dtype=uint8)

In [150]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.65      0.93      0.76      2814
           1       0.85      0.46      0.59      2567

    accuracy                           0.70      5381
   macro avg       0.75      0.69      0.68      5381
weighted avg       0.75      0.70      0.68      5381



### PCA

In [151]:
pca = PCA(3)
X_pca = pca.fit_transform(X)
pca.explained_variance_ratio_

array([0.14085962, 0.07924473, 0.06701291])

In [152]:
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(X_pca, y, test_size = 0.2, random_state=0, stratify=y)


clf = SVC(class_weight='balanced')
clf.fit(X_train_p, y_train_p)
y_pred = clf.predict(X_test_p)

print(classification_report(y_pred, y_test_p))

              precision    recall  f1-score   support

           0       0.68      0.93      0.79      2928
           1       0.85      0.48      0.61      2453

    accuracy                           0.72      5381
   macro avg       0.77      0.70      0.70      5381
weighted avg       0.76      0.72      0.71      5381

